REDEFINING THE MODEL AND IMPROVING ACCURACY BY IMPLEMENTING METHODS LIKE  THE HYPERPARAMETER TUNING, DATA PREPROCESSING, EARLY STOPPING TECHNIQUES.

This the model i worked for Assignment-2 this model could contribute to one of the model in dual model for my project. The Dual model will have two models for each gender the female one have pregenancies section but male section dont have tha feature here i preprocesse the data by eliminating the female rows and work on male rows so that this model may used.

In [2]:
#importing necessary libraries
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

In [3]:
#importing modified dataset
diabetes_data = pd.read_csv('/content/modified_diabetes_dataset.csv')

diabetes_data.head()

,Gender,Age,BMI,BloodPressure,Glucose,Outcome
0,1,50,33.6,72.0,148.0,1
1,0,31,26.6,66.0,85.0,0
2,1,32,23.3,64.0,183.0,1
3,1,21,28.1,66.0,89.0,0
4,1,33,43.1,40.0,137.0,1


To ready the model with only male values i am preprocessing the dataset to remove the female rows that encodes as  'Male'/'Female' to 0 and 1

In [4]:


# Change all values of 1 or 0 to 0 in the specified column
diabetes_data['Gender'] = diabetes_data['Gender'].apply(lambda x: 0 if x in [1, 0] else x)

# Optionally, save the modified dataset to a new file
diabetes_data.to_csv('modified_diabetes_dataset_males.csv', index=False)
diabetes_data.head()




,Gender,Age,BMI,BloodPressure,Glucose,Outcome
0,0,50,33.6,72.0,148.0,1
1,0,31,26.6,66.0,85.0,0
2,0,32,23.3,64.0,183.0,1
3,0,21,28.1,66.0,89.0,0
4,0,33,43.1,40.0,137.0,1


Changed the dataset Gender column to all males to have more data for sake of project


 Model building and training with new dataset

In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
!pip install keras_tuner
import keras_tuner as kt  # Keras Tuner for hyperparameter tuning


# Load your data
# diabetes_data = pd.read_csv('your_dataset.csv') # Uncomment and replace with actual data path
# Assuming diabetes_data is already loaded and preprocessed

# Features and target
X = diabetes_data.drop("Outcome", axis=1)  # Features: remove the 'Outcome' column
y = diabetes_data["Outcome"]  # Target: 'Outcome'

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# Define a function to build the model for Keras Tuner
def build_model(hp):
    model = Sequential()

    # Input layer
    model.add(Dense(
        units=hp.Int('units_input', min_value=8, max_value=32, step=8),
        activation='relu',
        input_shape=(X_train.shape[1],)
    ))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout_input', min_value=0.1, max_value=0.4, step=0.1)))

    # Hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            units=hp.Int(f'units_{i}', min_value=8, max_value=32, step=8),
            activation='relu'
        ))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.3, step=0.1)))

    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Set up Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of hyperparameter combinations to try
    directory='hyperparameter_tuning',
    project_name='diabetes_prediction'
)

# Early stopping callback
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Run the hyperparameter search
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Checkpoint callback to save the best model
checkpoint = ModelCheckpoint("best_optimized_model_tuned.keras", monitor="val_accuracy", save_best_only=True, mode="max")

# Train the best model
history = best_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])

# Load the best saved model
final_model = tf.keras.models.load_model("/content/best_optimized_model_tuned.keras")

# Evaluate on test data
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"Tuned Test Accuracy: {accuracy:.4f}")


Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.7705627679824829

Best val_accuracy So Far: 0.7878788113594055
Total elapsed time: 00h 01m 47s
Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5900 - loss: 0.7073 - val_accuracy: 0.6667 - val_loss: 0.6344
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6480 - loss: 0.6466 - val_accuracy: 0.7013 - val_loss: 0.6060
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6446 - loss: 0.5992 - val_accuracy: 0.7100 - val_loss: 0.5853
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7087 - loss: 0.5675 - val_accuracy: 0.7359 - val_loss: 0.5661
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7309 - loss: 0.5714 - val_accuracy: 0.7403 - val_loss: 0.5521
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7418 - loss: 0.5515 - val_accuracy: 0.7489 - val_loss: 0.5400
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7238 - loss: 0.5395 - val_acc